# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [7]:
import numpy as np

# Useful functions

In [21]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [14]:
def calc_rss(output,labels):
    residuals=output-labels
    rss=np.dot(residuals,residuals)
    return rss

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [2]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [3]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [4]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [8]:
coefs = model_all.coef_

In [9]:
np.nonzero(coefs)

(array([ 3, 10, 12], dtype=int64),)

In [10]:
len(all_features)

17

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

#### ANSWER

sqft_living, view, grade

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:

In [12]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [13]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. 
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

In [15]:
for l1 in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    
    predictions = model.predict(validation[all_features])
    print(calc_rss(predictions,validation['price']))

398213327300134.94
399041900253346.8
429791604072559.7
463739831045121.1
645898733633800.6
1222506859427163.0
1222506859427163.0
1222506859427163.0
1222506859427163.0
1222506859427163.0
1222506859427163.0
1222506859427163.0
1222506859427163.0


In [16]:
np.logspace(1, 7, num=13)

array([1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07])

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

ANSWER: 10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [17]:
model = linear_model.Lasso(alpha=10, normalize=True)
model.fit(training[all_features], training['price'])

Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [18]:
model.coef_

array([-1.61445628e+04,  3.73245384e+02,  5.08412433e+04,  6.17853560e+02,
       -4.44113549e+04,  7.85623065e-01, -7.01194765e+02, -0.00000000e+00,
        5.01420046e+03,  6.19488752e+05,  3.80418557e+04,  2.49987718e+04,
        1.28716235e+05,  0.00000000e+00,  0.00000000e+00, -3.29383118e+03,
        1.00573209e+01])

In [19]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

ANSWER: 15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [37]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [31]:
l1_penalty_values = np.logspace(1, 4, num=20)

In [47]:
l1_penalty_values

array([   10.        ,    14.38449888,    20.69138081,    29.76351442,
          42.81332399,    61.58482111,    88.58667904,   127.42749857,
         183.29807108,   263.66508987,   379.26901907,   545.55947812,
         784.75997035,  1128.83789168,  1623.77673919,  2335.72146909,
        3359.81828628,  4832.93023857,  6951.92796178, 10000.        ])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data.
    * Extract the weights of the model and count the number of nonzeros. (including intercept)

In [33]:
nonzeros_list=[]
for l1 in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    
    #print(model.coef_,model.intercept_)
    
    nonzeros=np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nonzeros_list.append(nonzeros)

In [34]:
nonzeros_list

[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [64]:
idx=0
while(nonzeros_list[idx]>max_nonzeros):
    idx+=1
#print(idx,nonzeros_list[idx])
l1_penalty_min=l1_penalty_values[idx-1]

In [65]:
l1_penalty_min

127.42749857031335

In [66]:
list(reversed(nonzeros_list))

[1, 1, 1, 1, 2, 3, 3, 5, 6, 6, 6, 7, 10, 11, 12, 13, 15, 15, 15, 15]

In [67]:
idx=0
while(list(reversed(nonzeros_list))[idx]<max_nonzeros):
    idx+=1
#print(idx,list(reversed(nonzeros_list))[idx])
l1_penalty_max=l1_penalty_values[-idx+1]

In [68]:
l1_penalty_max

379.26901907322497

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

ANSWER = 127.42749857031335 ,379.26901907322497

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [71]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [73]:
l1_penalty_values

array([127.42749857, 140.68231544, 153.93713231, 167.19194918,
       180.44676604, 193.70158291, 206.95639978, 220.21121665,
       233.46603352, 246.72085039, 259.97566726, 273.23048412,
       286.48530099, 299.74011786, 312.99493473, 326.2497516 ,
       339.50456847, 352.75938534, 366.0142022 , 379.26901907])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. 
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [72]:
for l1 in l1_penalty_values:
    model = linear_model.Lasso(alpha=l1, normalize=True)
    model.fit(training[all_features], training['price'])
    
    predictions = model.predict(validation[all_features])
    
    nonzeros=np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    
    print(calc_rss(predictions,validation['price']),nonzeros)

435374677102680.7 10
438102373525174.3 8
439824336123779.94 7
441201860565516.06 7
442751308216660.94 7
444475708047902.56 7
446344227588950.7 6
447943750743218.1 6
449684227573327.8 6
451566330926753.1 6
453588680430141.2 6
455751983516286.4 6
458056251657879.25 6
460501487147748.4 6
463087679119237.25 6
465814831790699.75 6
468682938297670.0 6
471691998640148.0 6
474842012818133.7 6
478132980831626.94 6


In [74]:
model = linear_model.Lasso(alpha=153.93713231, normalize=True)
model.fit(training[all_features], training['price'])

Lasso(alpha=153.93713231, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [77]:
all_features

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [76]:
model.coef_

array([-0.00000000e+00, -0.00000000e+00,  1.10651889e+04,  1.63237910e+02,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  5.08182019e+05,  4.19971947e+04,  0.00000000e+00,
        1.16480285e+05,  0.00000000e+00,  0.00000000e+00, -2.62802944e+03,
        0.00000000e+00])

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

#### ANSWERS

1: 153.93713231

2: bathrooms, sqft_living, waterfront, view, grade, yr_built